In [3]:
import numpy as np

In [6]:
x = np.array([[7.37 * 1e9, 7.42 * 1e9], [0.005, 0.07], [4.8 * 1e-08, 7.96 * 1e-07], [7.37 * 1e9, 7.42 * 1e9], [0.005, 0.07], [4.8 * 1e-08, 7.96 * 1e-07]])
y = x[:,1]
print(y)

[7.42e+09 7.00e-02 7.96e-07 7.42e+09 7.00e-02 7.96e-07]
